In [118]:
import pandas as pd
import numpy as np

In [119]:
DataDF = pd.read_csv('C:\Git\KDT\BigData\Project\DL\Motor_Vehicle_Theft.csv')

In [120]:
DataDF.columns

Index(['Unnamed: 0', 'ID', 'Case Number', 'Date', 'Block', 'IUCR',
       'Primary Type', 'Description', 'Location Description', 'Arrest',
       'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [121]:
## 전처리 : 결측치 올 제거
DataDF = DataDF.dropna()

## 전처리 : 칼럼 정리
DataDF = DataDF.loc[:,['Date','IUCR','Location Description','Arrest','District','Ward','Community Area']]

## IUCR 세부화 변경
label_dict= {910:['AUTOMOBILE',	1,	0,	0],
915	: ['TRUCK, BUS, MOTOR HOME',1,	0,	0],
917	: ['CYCLE, SCOOTER, BIKE',	1,	0,	0],
918	: ['CYCLE, SCOOTER, BIKE',	0,	0,	0],
920	: ['AUTOMOBILE',	1,	1,	0],
925	: ['TRUCK, BUS, MOTOR HOME',	1,	1,	0],
927	: ['CYCLE, SCOOTER, BIKE',	1,	1,	0],
928	: ['CYCLE, SCOOTER, BIKE',	0,	1,	0],
930	: ['AUTOMOBILE',	1,	0,	1],
935	: ['TRUCK, BUS, MOBILE HOME',	1,	0,	1],
937	: ['CYCLE, SCOOTER, BIKE',	1,	0,	1],
938	: ['CYCLE, SCOOTER, BIKE',0,0,1]}


DataDF['VIN']=DataDF['IUCR']
DataDF['ATTEMPT']=DataDF['IUCR']
DataDF['RECOVERY']=DataDF['IUCR']

for idx, dict in enumerate(label_dict):
    DataDF['IUCR'] = DataDF['IUCR'].replace(dict, label_dict[dict][0])
    DataDF['VIN'] = DataDF['VIN'].replace(dict, label_dict[dict][1])
    DataDF['ATTEMPT'] = DataDF['ATTEMPT'].replace(dict, label_dict[dict][2])
    DataDF['RECOVERY'] = DataDF['RECOVERY'].replace(dict, label_dict[dict][3])


## 전처리 1 : 시간 24시간화
DataDF['Date'][DataDF['Date'].str[-2:]=="AM"] = DataDF['Date'].str[-11:-9]
PMDF = DataDF['Date'][DataDF['Date'].str[-2:]=="PM"].str[-11:-9]
PMDF = PMDF.astype("int64")+12
DataDF['Date'][DataDF['Date'].str[-2:]=="PM"] = PMDF

C:\Users\kjy19\AppData\Local\Temp\ipykernel_19264\2528296466.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataDF['Date'][DataDF['Date'].str[-2:]=="AM"] = DataDF['Date'].str[-11:-9]
C:\Users\kjy19\AppData\Local\Temp\ipykernel_19264\2528296466.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataDF['Date'][DataDF['Date'].str[-2:]=="PM"] = PMDF


In [122]:
## 로케이션 압축화
# 거리
# STREET = ["SIDEWALK", "STREET", "ALLEY", "DRIVEWAY - RESIDENTIAL", 
#         "BRIDGE", "VACANT LOT / LAND", "VACANT LOT/LAND"]

Residential_Roads = ["SIDEWALK", "DRIVEWAY - RESIDENTIAL"]      # 주거지 도로

Commercial_Roads = ["STREET", "ALLEY"]  # 상업/도시 도로

Public_Spaces =["VACANT LOT / LAND","VACANT LOT/LAND", "BRIDGE"]  # 공공장소

HIGHWAY = ["HIGHWAY/EXPRESSWAY", "HIGHWAY / EXPRESSWAY"]

# 차고, 주차장, 부지
PARKINGZONE = ["PARKING LOT / GARAGE (NON RESIDENTIAL)",
        "CHA PARKING LOT / GROUNDS","CHA PARKING LOT/GROUNDS","PARKING LOT/GARAGE(NON.RESID.)",
        "RESIDENCE - GARAGE","CTA PARKING LOT / GARAGE / OTHER PROPERTY", 
        "CTA TRACKS - RIGHT OF WAY", "CTA GARAGE / OTHER PROPERTY", "OTHER RAILROAD PROPERTY / TRAIN DEPOT", 
        "OTHER RAILROAD PROP / TRAIN DEPOT", "WAREHOUSE"]

# 거주지
HOUSE = [ "RESIDENCE", "RESIDENCE - PORCH / HALLWAY", "RESIDENCE - YARD (FRONT / BACK)", 
         "APARTMENT", "RESIDENTIAL YARD (FRONT/BACK)", 
        "RESIDENCE-GARAGE", "ABANDONED BUILDING", "RESIDENCE PORCH/HALLWAY", "CHA APARTMENT"]

# 가게 ex) 백화점, 주유소, 호텔, 모텔, 세탁소, 은행, 영화관, 당구장, 경기장, 카지노 ...
STORE = [ "COMMERCIAL / BUSINESS OFFICE", "RESTAURANT", "GAS STATION", "CONVENIENCE STORE", 
        "SMALL RETAIL STORE", "DEPARTMENT STORE", "FACTORY / MANUFACTURING BUILDING", 
        "GROCERY FOOD STORE", 
        "BARBERSHOP", "APPLIANCE STORE", "NEWSSTAND", 
        "DRUG STORE",  "CLEANING STORE", "AUTO / BOAT / RV DEALERSHIP", 
        "CAR WASH", "PAWN SHOP"]



Alcohol = ["BAR OR TAVERN","TAVERN/LIQUOR STORE","CASINO/GAMBLING ESTABLISHMENT","TAVERN / LIQUOR STORE","TAVERN/LIQUOR STORE","BAR OR TAVERN"]


# 숙박시설 
Accommodation = ["HOTEL/MOTEL","HOTEL / MOTEL",]

# 유흥, 스포츠
SPORTS = ["SPORTS ARENA / STADIUM","POOL ROOM", "MOVIE HOUSE / THEATER","SPORTS ARENA/STADIUM","BOWLING ALLEY",
          "ATHLETIC CLUB","MOVIE HOUSE/THEATER"]


# 은행, 금융관련
BANK = ["ATM (AUTOMATIC TELLER MACHINE)","CURRENCY EXCHANGE","BANK"]
# 차량 ex) 버스, 기차, 트럭 ...
CAR = ["CTA TRAIN", "CTA BUS",  
        "VEHICLE NON-COMMERCIAL", "VEHICLE - COMMERCIAL", "VEHICLE - DELIVERY TRUCK", 
        "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)", "TAXICAB", 
        "VEHICLE-COMMERCIAL", "VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS", 
        "VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS", "DELIVERY TRUCK", "OTHER COMMERCIAL TRANSPORTATION",
        "VEHICLE - OTHER RIDE SERVICE", "VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)", 
        "VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)"]
# 정류장
STATION = ["CTA PLATFORM", "CTA STATION", "CTA BUS STOP"]

# 종료
CHURCH = ["CHURCH / SYNAGOGUE / PLACE OF WORSHIP", "SYNAGOGUE", "CHURCH", "CHURCH/SYNAGOGUE/PLACE OF WORSHIP"]

# 학교
SCHOOL = ["SCHOOL, PUBLIC, BUILDING", "SCHOOL - PUBLIC GROUNDS", "SCHOOL, PUBLIC, GROUNDS", 
        "SCHOOL - PRIVATE BUILDING", "SCHOOL - PRIVATE GROUNDS", "SCHOOL, PRIVATE, BUILDING", 
        "SCHOOL - PUBLIC BUILDING", "SCHOOL, PRIVATE, GROUNDS", "COLLEGE / UNIVERSITY - GROUNDS", 
        "COLLEGE/UNIVERSITY GROUNDS", "COLLEGE/UNIVERSITY RESIDENCE HALL"]

# 공원 ex) 호수, 자연보호구역 ? , 보트/수상기구 ?
PARK = [ "PARK", "PARK PROPERTY", "LAKEFRONT / WATERFRONT / RIVERBANK", "FOREST PRESERVE", 
        "CEMETARY", "BOAT / WATERCRAFT", "BOAT/WATERCRAFT", "LAKEFRONT/WATERFRONT/RIVERBANK",
        "LAKEFRONT/WATERFRONT/RIVERBANK"]

# 병원
HOSPITAL = ["HOSPITAL", "HOSPITAL BUILDING / GROUNDS", "MEDICAL/DENTAL OFFICE", "NURSING HOME/RETirement Home", 
        "ANIMAL HOSPITAL", "HOSPITAL BUILDING/GROUNDS", "MEDICAL / DENTAL OFFICE", "NURSING HOME/RETIREMENT HOME",
        "NURSING / RETIREMENT HOME"]

# 공항
AIRPORT = ["AIRPORT", "AIRPORT TERMINAL UPPER LEVEL - SECURE AREA", "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA", 
        "AIRPORT PARKING LOT", "AIRPORT BUILDING NON-TERMINAL - SECURE AREA", 
        "AIRPORT EXTERIOR - SECURE AREA", "AIRPORT/AIRCRAFT", "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA", 
        "AIRPORT EXTERIOR - NON-SECURE AREA", "AIRPORT VENDING ESTABLISHMENT", 
        "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA","AIRPORT TRANSPORTATION SYSTEM (ATS)",
        "AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA",'AIRCRAFT']

# 경찰서
POLICE = ["POLICE FACILITY / VEHICLE PARKING LOT", "POLICE FACILITY/VEH PARKING LOT"]

# 정부 
GOVERNMENT = ["GOVERNMENT BUILDING / PROPERTY", "FEDERAL BUILDING", "GOVERNMENT BUILDING/PROPERTY"]

# 소방서
FIRE_STATION = ["FIRE STATION"]

# 교도소 
JAIL = ["JAIL / LOCK-UP FACILITY", "LOCK-UP FACILITY"]

# 공장 ex) 건설현장
FACTORY = ["FACTORY/MANUFACTURING BUILDING", "CONSTRUCTION SITE"]

# 엘리베이터
ELEVATOR = ["CHA HALLWAY / STAIRWELL / ELEVATOR", "CHA HALLWAY/STAIRWELL/ELEVATOR"]

# 유치원 
kindergarten = ["DAY CARE CENTER"]

# 도서관 
LIBRARY = ["LIBRARY"]


# 기타
OTHER = ["OTHER", "OTHER (SPECIFY)", "SAVINGS AND LOAN", "COIN OPERATED MACHINE"]

location = [Residential_Roads, Commercial_Roads,Public_Spaces,PARKINGZONE,HOUSE,STORE,CAR,STATION,CHURCH,SCHOOL,PARK,HOSPITAL,AIRPORT,POLICE,
            GOVERNMENT,FIRE_STATION,FACTORY, JAIL,
            Accommodation, BANK,ELEVATOR,kindergarten,LIBRARY,OTHER,Alcohol,HIGHWAY,SPORTS]
loc_name = ['Residential_Roads', 'Commercial_Roads','Public_Spaces','PARKINGZONE','HOUSE','STORE','CAR','STATION','CHURCH','SCHOOL','PARK','HOSPITAL','AIRPORT','POLICE','GOVERNMENT','FIRE_STATION',
            'FACTORY','ELEVATOR','kindergarten','LIBRARY','OTHER','Accommodation', 'BANK','Alcohol',"HIGHWAY","SPORTS","JAIL"]

len(location)

idx2 = 0
for lo in location:
    for idx in range(len(lo)):
        DataDF['Location Description'] = DataDF['Location Description'].replace(lo[idx],loc_name[idx2])
    idx2+=1


In [127]:
len(loc_name)

27

In [123]:
print("---- 차량번호 , 시도, 회수 비율 검색-----")
print(DataDF['VIN'].value_counts())
print(DataDF['ATTEMPT'].value_counts())
print(DataDF['RECOVERY'].value_counts())

---- 차량번호 , 시도, 회수 비율 검색-----
VIN
1    375235
0       346
Name: count, dtype: int64
ATTEMPT
0    353450
1     22131
Name: count, dtype: int64
RECOVERY
0    346898
1     28683
Name: count, dtype: int64


In [124]:
print("'Location Description' : ", DataDF['Location Description'].unique())
print("'Location Description' unique : ",len(DataDF['Location Description'].unique()))
print("'Location Description' 비율 \n",DataDF['Location Description'].value_counts())

'Location Description' :  ['Residential_Roads' 'Commercial_Roads' 'PARKINGZONE' 'STORE' 'HOUSE'
 'CAR' 'Alcohol' 'SCHOOL' 'STATION' 'Public_Spaces' 'POLICE'
 'Accommodation' 'SPORTS' 'FACTORY' 'GOVERNMENT' 'AIRPORT' 'HOSPITAL'
 'PARK' 'kindergarten' 'JAIL' 'BANK' 'CHURCH' 'HIGHWAY' 'FIRE_STATION'
 'ELEVATOR']
'Location Description' unique :  25
'Location Description' 비율 
 Location Description
Commercial_Roads     299530
PARKINGZONE           30854
HOUSE                 13731
Alcohol                7984
STORE                  6876
Residential_Roads      5384
CAR                    4400
Public_Spaces          2390
AIRPORT                1102
SCHOOL                  697
PARK                    517
POLICE                  470
JAIL                    436
kindergarten            334
HOSPITAL                296
CHURCH                  140
FACTORY                 134
GOVERNMENT               85
HIGHWAY                  73
SPORTS                   65
BANK                     34
STATION         

In [125]:
DataDF.head(5)

,Date,IUCR,Location Description,Arrest,District,Ward,Community Area,VIN,ATTEMPT,RECOVERY
0,22.0,AUTOMOBILE,Residential_Roads,False,4.0,10.0,52.0,1,0,0
1,08,AUTOMOBILE,Commercial_Roads,False,19.0,44.0,6.0,1,0,0
4,12,AUTOMOBILE,PARKINGZONE,False,4.0,7.0,43.0,1,0,0
5,14.0,AUTOMOBILE,Commercial_Roads,False,2.0,3.0,38.0,1,0,1
6,16.0,AUTOMOBILE,STORE,False,6.0,6.0,69.0,1,0,0


In [126]:
DataDF.to_csv('Data.csv')